# Topics in Computer Science - Bitcoin: Programming the Future of Money - ITCS 4010 & 5010 - Spring 2025 - UNC Charlotte

# Homework 4 - Digital Signatures in Bitcoin (100 Points)

# <font color="blue"> Submission instructions</font>

1. Click the Save button at the top of the Jupyter Notebook.
2. Please make sure to have entered your name above.
3. Select Cell -> All Output -> Clear. This will clear all the outputs from all cells (but will keep the content of all cells). 
4. Select Cell -> Run All. This will run all the cells in order, and will take several minutes.
5. Once you've rerun everything, create a PDF version of the executed Jupyter Notebook via "File" -> "Download As" and then choosing on of the options "PDF via LaTeX", "PDF via HTML" or "HTML" and download a PDF or HTML version showing the code and the output of all cells. If you download a HTML version, you can print this HTML file as a PDF in a second step. Save the PDF version in the same folder that contains the notebook file.
6. Look at the PDF file and make sure all your solutions are there, displayed correctly.
7. Submit **both** your PDF and the notebook file .ipynb on Gradescope.
8. Make sure your your Gradescope submission contains the correct files by downloading it after posting it on Gradescope.

In [2]:
class FieldElement:

    def __init__(self, num, prime):
        #check if 0 > num >= prime. Raise ValueError if num is out of range.
        if num >= prime or num < 0:
            error = 'Num {} not in field range 0 to {}'.format(num, prime - 1)
            raise ValueError(error)
        #Initialize num and prime
        self.num = num
        self.prime = prime

    def __repr__(self):
        return 'FieldElement_{}({})'.format(self.prime, self.num)

    def __eq__(self, other):
        if other is None:
            return False
        #Return True if the FieldElement objects are equal
        return self.num == other.num and self.prime == other.prime

    def __ne__(self, other):
        # This should be the inverse of the == operator
        return not (self == other)

    def __add__(self, other):
        # Two numbers have to be in same field, otherwise raise error
        if self.prime != other.prime:
            raise TypeError('Cannot add two numbers in different Fields')
        #Perform addition of two finite field elements
        num = (self.num + other.num) % self.prime
        # Return an element of the same class
        return self.__class__(num, self.prime)

    def __sub__(self, other):
        # Two numbers have to be in same field, otherwise raise error
        if self.prime != other.prime:
            raise TypeError('Cannot subtract two numbers in different Fields')
        #Perform subtraction of two finite field elements
        num = (self.num - other.num) % self.prime
        return self.__class__(num, self.prime)

    def __mul__(self, other):
        # Two numbers have to be in same field, otherwise raise error
        if self.prime != other.prime:
            raise TypeError('Cannot multiply two numbers in different Fields')
        # Perform muliplication of two finite field elements
        num = (self.num * other.num) % self.prime
        return self.__class__(num, self.prime)

    def __pow__(self, exponent):
        #Implement finite field exponentation
        n = exponent % (self.prime - 1)
        num = pow(self.num, n, self.prime)
        return self.__class__(num, self.prime)

    def __truediv__(self, other):
        # Two numbers have to be in same field, otherwise raise error
        if self.prime != other.prime:
            raise TypeError('Cannot divide two numbers in different Fields')
        # perform division of two finite field elements
        # Hint: Use fermat's little theorem:
        num = (self.num * pow(other.num, self.prime - 2, self.prime)) % self.prime
        return self.__class__(num, self.prime)

    def __rmul__(self, coefficient):
        # Implement scalar multiplication: Multiply the scalar 'coeffiecient' with finite field element.
        num = (self.num * coefficient) % self.prime
        return self.__class__(num=num, prime=self.prime)

In [3]:
class Point:

    def __init__(self, x, y, a, b):
        self.a = a
        self.b = b
        self.x = x
        self.y = y
        if self.x is None and self.y is None:
            return
        if self.y**2 != self.x**3 + a * x + b:
            # if not, throw a ValueError
            raise ValueError('({}, {}) is not on the curve'.format(x, y))

    def __eq__(self, other):
        return self.x == other.x and self.y == other.y \
            and self.a == other.a and self.b == other.b

    def __ne__(self, other):
        # this should be the inverse of the == operator
        return not (self == other)

    def __repr__(self):
        if self.x is None:
            return 'Point(infinity)'
        elif isinstance(self.x, FieldElement):
            return 'Point({},{})_{}_{} FieldElement({})'.format(
                self.x.num, self.y.num, self.a.num, self.b.num, self.x.prime)
        else:
            return 'Point({},{})_{}_{}'.format(self.x, self.y, self.a, self.b)

    def __add__(self, other):
        if self.a != other.a or self.b != other.b:
            raise TypeError('Points {}, {} are not on the same curve'.format(self, other))
        
        if self.x is None:
            return other
        if other.x is None:
            return self

        if self.x == other.x and self.y != other.y:
            return self.__class__(None, None, self.a, self.b)

        if self.x != other.x:
            s = (other.y - self.y) / (other.x - self.x)
            x = s**2 - self.x - other.x
            y = s * (self.x - x) - self.y
            return self.__class__(x, y, self.a, self.b)

        if self == other and self.y == 0 * self.x:
            return self.__class__(None, None, self.a, self.b)

        if self == other:
            s = (3 * self.x**2 + self.a) / (2 * self.y)
            x = s**2 - 2 * self.x
            y = s * (self.x - x) - self.y
            return self.__class__(x, y, self.a, self.b)

    def __rmul__(self, coefficient):
        coef = coefficient
        current = self
        result = self.__class__(None, None, self.a, self.b)
        while coef:
            if coef & 1:
                result += current
            current += current
            coef >>= 1
        return result


The classes `S256Field` and `S256Point` are subclasses of `FieldElement` and `Point`, respectively, specifically designed to work with the parameters of secp256k1. These subclasses simplify the process of initializing a point on the secp256k1 curve by eliminating the need to repeatedly define the curve parameters `a` and `b`, as required when using the Point class.

In [4]:
Prime = 2**256 - 2**32 - 977 # this is the prime that determines the size of the finite field F_p on which the secp256k1 points live
class S256Field(FieldElement):

    def __init__(self, num, prime=None):
        super().__init__(num=num, prime=Prime)

    def __repr__(self):
        return '{:x}'.format(self.num).zfill(64)

In [5]:
A = 0
B = 7
n = 0xfffffffffffffffffffffffffffffffebaaedce6af48a03bbfd25e8cd0364141 # order of group generated by generator point G

class S256Point(Point):

    def __init__(self, x, y, a=None, b=None):
        a, b = S256Field(A), S256Field(B)
        if type(x) == int:
            super().__init__(x=S256Field(x), y=S256Field(y), a=a, b=b)
        else:
            super().__init__(x=x, y=y, a=a, b=b)  # <1>

    def __repr__(self):
        if self.x is None:
            return 'S256Point(infinity)'
        else:
            return 'S256Point({}, {})'.format(self.x, self.y)

    def __rmul__(self, coefficient):
        coef = coefficient % n
        return super().__rmul__(coef)

G = S256Point(0x79be667ef9dcbbac55a06295ce870b07029bfcdb2dce28d959f2815b16f81798,
    0x483ada7726a3c4655da4fbfc0e1108a8fd17b448a68554199c47d08ffb10d4b8)

The generator point `G` on the secp256k1 curve used in Bitcoin is defined as follows:

$G = (G_x, G_y)$



In [6]:
G = S256Point(0x79be667ef9dcbbac55a06295ce870b07029bfcdb2dce28d959f2815b16f81798,
    0x483ada7726a3c4655da4fbfc0e1108a8fd17b448a68554199c47d08ffb10d4b8)

In [7]:
import hashlib
def hash256(s):
    '''two rounds of sha256'''
    return hashlib.sha256(hashlib.sha256(s).digest()).digest()

### 1. ECDSA
#### a. 
Write a function `ecdsa_sign` that implements the signing function of ECDSA on the secp256k1 elliptic curve with generator (or base) point `G` from above, and computes and returns the ECDSA signature pair $(r,s)$ for the given private key `e`, message `m` and private nonce `k`. Use `hash256` from above as the hash function $\operatorname{hash}(\cdot)$ of the scheme.

In [8]:
def ecdsa_sign(e, m, k):
    m = m.encode()
    # Compute R = KG
    R = (k*G)
    # Calculate r (R's x-coordinate)
    r = R.x.num
    #Compute z = sha256(sha256(m))
    z = int.from_bytes(hash256(m), 'big')
    # Calculate s = (z+re)/k
    k_inv = pow(k, N-2, N)
    s = (z+r*e) * k_inv % N
    return (r,s)

#### b.
Write a function `ecdsa_verify` to verify if the generated signature `s` is valid or not using the public key `P`, message `m`, x coordinate of the public nonce `r`. Return `True` if signature is valid, `False` otherwise.

In [9]:
def ecdsa_verify(P, m, r, s):
    m = m.encode()
    z = int.from_bytes(hash256(m), 'big')
    s_inv = pow(s, N-2, N)
    u = z * s_inv % N
    v = r * s_inv % N
    return (u * G + v * P).x.num == r

#### Testcases

Verfiy the workings of the the functions implemented in *a.* and *b.* above using the messages `m1`, `m2` defined below and the random private nonces `k1` and `k2` defined below, each for the private key `e`.

**Generate the output of `schnorr_sign` in the cases 1.-4. from Exercise 1. "ECDSA" above, and print this output successively.**


In [10]:
Prime = 2**256 - 2**32 - 977
N = 0xfffffffffffffffffffffffffffffffebaaedce6af48a03bbfd25e8cd0364141 #Group order
e = 246835 #private key
# messages
m1 = 'Bitcoin'
m2 = 'Lightning'


In [11]:
import random
random.seed(10)
k1 = random.randint(0,n) # private nonce k1
print(k1)
random.seed(100)
k2 = random.randint(0,n) # private nonce k2
print(k2)

53563162965150739642758196075283649051494876524252451189941645365582255059155
45510187412815177975571940105204457996594329113615379591172816318882246202515


**Note:** It is *unsafe* to use the pseudo-random number generator of the Python module `random` in production software such as for generating your own private key or random nonce to be used on the actual Bitcoin network, see [discussion here](https://docs.python.org/3/library/random.html).

**Generate the output of `ecdsa_sign` in  the following cases, and print this output successively:**

1. For the message `m1` and private nonce `k1`
2. For `m2` and `k1`
3. For `m1` and `k2`
4. For `m2` and `k2`

(Print your answers for all the above cases)

In [12]:
P = e*G
(r1, s11) = ecdsa_sign(e, m1, k1)
print((r1,s11))

(17475157730576101221922219781495314687234679970816515184941578315767911417842, 19660279264506430846191789191043658456529652824310696416138487341954076124048)


In [13]:
(r1, s21) = ecdsa_sign(e, m2, k1)
print((r1,s21))

(17475157730576101221922219781495314687234679970816515184941578315767911417842, 33322851105363613992326303623689941992523767831842844914406511441389261349046)


In [14]:
(r2, s12) = ecdsa_sign(e, m1, k2)
print((r2,s12))

(13614136033041484195754855738883558962512156503117353260198194967466378178770, 102434461432272869824313281266879855732702769532194605636940878443543330997803)


In [15]:
(r2, s22) = ecdsa_sign(e, m2, k2)
print((r22,s22))

NameError: name 'r22' is not defined

**Then, verify the generated signatures for the four cases 1.-4. using `ecdsa_verify`, and print the output.**


In [16]:
print(ecdsa_verify(P, m1, r1, s11))
print(ecdsa_verify(P, m2, r1, s21))
print(ecdsa_verify(P, m1, r2, s12))
print(ecdsa_verify(P, m2, r2, s22))

True
True
True
True


**c. What is the result of the signature verification of the output of setup 1. using the public nonce of the setup of 3.?** Explain your result.

In [17]:
print(ecdsa_verify(P, m1, r2, s11))

False


Setup 1 and 3 both try to create a signature of for the same message (`m1`) using the same private-public key pair (`e`,`P`). However, they are using a different private nonce (`k1` and `k2`), respectively, which leads to different public nonces `r1` and `r2`. Therefore, using the a pair of public nonce r and signatures s that do not match leads to a `False` result in the `ecdsa_verify` function. 

#### d. (Multiple signatures for same nonce, message and key?)
Assume that $(r,s)$ is a valid ECDSA signature for the message `m`, private key `e`, and private nonce `k`.

Recall that $n$ is the order of the group $\{j G \in S_{0,7}: j$ is a positive integer$\}$ generated by the generator point $G$. <br>
**Show that then $(r,n−s)$ is also a valid signature for the same message, key and nonce.**

(**Hint:** You will not be able to solve this problem by coding. If you are not familiar with writing equations and formula in LaTeX/Markdown, we suggest that you write down your solution on a paper, take a picture/scan and insert the scan below.)

In [18]:
e = 12345 #private key
# messages
m = 'Bitcoin'
# Private nonce
k = 987654321
P = e*G
(r, s) = ecdsa_sign(e, m, k)
print(ecdsa_verify(P, m, r, s))
print(ecdsa_verify(P, m, r, N-s))

True
True


In [19]:
print(s)

104631914286407385980319307529796271411987572269560988302605005079211003539381


In [20]:
print(N-s)

11160174950908809443251677478891636440849992009513916080000158062307157954956


#### e. (ECDSA Signature Length Reduction)
**Explain how the statement from *d.* above can be used to reduce the maximum length of a DER-encoded ECDSA signature used in the Bitcoin blockchain from 73 bytes to 72 bytes.**

Including the SIGHASH flag, DER-encoded ECDSA signatures used in the Bitcoin blockchain can have up to 73 bytes of length (Lecture 13) in general. The DER encoding rules state that:
    
    Encode s as a (signed) big-endian integer, but prepend with the 0x00 byte if s’s first byte ≥ 0x80. Prepend the resulting length to s. Add this to the result.
    
The encoding of $s$ thus includes an additional byte `0x00` if the value of $s$ is larger than $2^{256} /2$. This byte will be prevented to appear in the DER encoding of the signature if, among $s$ and $n-s$, the smaller one is chosen.

#### f. (Public Key Recovery)
Assume you are given a message `m` and the output (`r`,`s`) of an ECDSA signing function, but you do not know the underlying private key `e`, neither do you know the private nonce `k`. Furthermore, you are even not provided with the public key `P` that corresponds to this signing function.

I claim that from this information, it is possible to "narrow" down what this public key `P` to at least two different possible options `P1` and `P2`, so that very likely, either `P == P1` or `P == P2`.

**Show below what these two options `P1` and `P2` are.**

Let $z = \operatorname{hash}($ `m` $ )$ if `m` is the message. From the condition checked in `ecdsa_verify` we see that the $x$-coordinate of
$$
u G + v P = \frac{z}{s}G + \frac{r}{s} P
$$
is supposed to coincide with $r=$ `r`. Let $R = (r,r_y)$ be the elliptic curve point obtained by taking $r_y = \sqrt{r^3 + 7}$ (see "how to derive $y$-coordinate from $x$-coordinate in Compressed SEC" in Lecture 13 for how to compute this square root explicitly). <br>
**Case 1:** It holds that $R = \frac{z}{s}G + \frac{r}{s} P$. <br>
In this case, we can solve for $P$ as follows:
\begin{align*}
R &= s^{-1}(zG + rP)  \\ 
sR &= zG + rP  \\
rP &= sR - zG  \\
 P &= r^{-1}(sR - zG)  \\
\end{align*}
In the above equation, the inverse $r^{-1}$ of $r$ is taken within $F_n$, where $n$ is the order of the group generated by the generator point $G$.

Alternatively, a second case can be true: <br>
**Case 2:** It holds that $-R = \frac{z}{s}G + \frac{r}{s} P$. <br>
Similarly to above, we can rearrange this to
\begin{align*}
-R &= s^{-1}(zG + rP)  \\ 
-sR &= zG + rP  \\
rP &= -sR - zG  \\
 P &= r^{-1}(-sR - zG)  \\
\end{align*}

Since we are only given the $x$-coordinate $r$ of $R$, either case could be true, so `P1`=$= r^{-1}(sR - zG)$ and `P2`$= r^{-1}(-sR - zG)$ are two options for the public key. that corresponds to the given information.

#### g. (BONUS QUESTION: Public Key Recovery, 20 bonus points)

**Note:** Answering this question is not required, but can provide you with bonus points to the homework assignment.

In fact, in the setup of *f.* above, there might be actually **four** possibilities for the public key `P` to be recovered (among which two are very unlikely). **Describe what the two unlikely possibilities for the public key `P` are.**

We recall that scalars of elliptic curve points (such as $s$) are in $F_n$, whereas coordinates of elliptic curve points are in $F_p$, where $p$ is the prime order of the finite field ($p= 2**256 - 2**32 - 977$ for secp256k1). $n$ is of the same order of magnitude as $p$ (`n = 0xfffffffffffffffffffffffffffffffebaaedce6af48a03bbfd25e8cd0364141` vs. `p = 0xfffffffffffffffffffffffffffffffffffffffffffffffffffffffefffffc2f` in  in hexadecimal), but it holds that $n < p$.

In the case of very small values of $r \in F_n$, it could be that the associated public nonce point is 
$$R_3 = (r + n, \sqrt{(r+n)^3 + 7}),$$
or, 
$$R_4 = (r + n, -\sqrt{(r+n)^3 + 7}),$$
respectively.
This leads to the possibilities for the public keys to be `P3`=$= r^{-1}(sR_3 - zG)$ or `P4`=$= r^{-1}(sR_4 - zG)$.

However, we note that these cases cannot occur (or rather, coincide with `P1` and `P2` above) unless $(r +n) \operatorname{mod} p \neq (r +n) \operatorname{mod} n$. This is only the case if  $0 < r < p - n$, which occurs approximately with probability $\frac{p-n}{n} \approx 3.7345\cdot 10^{-39}$ (over the choice of a random private nonce $k$), which is very very small.  

### 2. Schnorr Signatures (40 Points)

In the [Taproot update of the Bitcoin protocol](https://github.com/bitcoin/bips/blob/master/bip-0341.mediawiki), which was introduced as a [soft fork](https://en.wikipedia.org/wiki/Fork_(blockchain)) in August 2021, a new digital signature scheme was introduced in a new address format, which is based on [Schnorr signatures](https://github.com/bitcoin/bips/blob/master/bip-0340.mediawiki).

#### a.
Write a function `schnorr_sign` that computes and returns Schnorr signature and public nonce pair $(s, R)$ for the given private key `e`, message `m` and private nonce `k`.

(**Hint:** You can use the version of the Schnorr signature scheme that has been discussed in class (R is returned as an (uncompressed) elliptic curve point); a strict adherence to the serialization of BIP 340 is not necessary.)

In [53]:
def schnorr_sign(e, m, k):
    R = k*G
    P = e*G
    R_xonly = R.x.num.to_bytes(32, 'big')
    P_xonly = P.x.num.to_bytes(32, 'big')
    m = m.encode()
    z = int.from_bytes(hash256(R_xonly + P_xonly + m), 'big') % N
    s = (k + z*e) % N
    return (s, R)

#### b.
Write a function `schnorr_verify` to verify if the generated schnorr signature `s` is valid or not using the public key `P`, message `m`, x coordinate of the public nonce `r`. Return `True` if signature is valid, `False` otherwise.

In [54]:
def schnorr_verify(P, m, s, R):
    P_xonly = P.x.num.to_bytes(32, 'big')
    R_xonly = R.x.num.to_bytes(32, 'big')
    m = m.encode()
    z = int.from_bytes(hash256(R_xonly + P_xonly + m), 'big') % N
    target = z*P + R
    return target == s*G

#### Testcases

**Generate the output of `schnorr_sign` in the cases 1.-4. from Exercise 1. "ECDSA" above, and print this output successively.**

Verfiy the workings of the the functions implemented in *a.* and *b.* above using the messages `m1`, `m2` defined below and the random private nonces `k1` and `k2` defined below, each for the private key `e`.

Generate signatures for the following cases:
1. For the message `m1` and private nonce `k1`
2. For `m2` and `k1`
3. For `m1` and `k2`
4. For `m2` and `k2`

Also, Verify the generated signatures in each case.

(Print your answers for all the above cases)

In [56]:
P = e*G
(s11, R1) = schnorr_sign(e, m1, k1)
print((s11,R1))

(19433819383232933904554569276503717832929153680935985500499855567107670990681, S256Point(26a296b96285ff81f10721ee78d3a3cea2f66e85220eac5fd60c9321aaf207f2, acc9a985c4e25b1d69c1d491633e3407c25c491c09445b70290368cbecccb829))


In [57]:
(s12, R1) = schnorr_sign(e, m2, k1)
print((s12,R1))

(78146440728767369034061725880925276201589829893812490721339413417531144024687, S256Point(26a296b96285ff81f10721ee78d3a3cea2f66e85220eac5fd60c9321aaf207f2, acc9a985c4e25b1d69c1d491633e3407c25c491c09445b70290368cbecccb829))


In [58]:
(s21, R2) = schnorr_sign(e, m1, k2)
print((s21,R2))

(63906732418935910857572104945439580557798766310031731787931150638898440201174, S256Point(1e1953f319bc2753b47231caaa25061bc6dc29b017eb01a848809ba70b12b0d2, 7aee36a5d36d5ad5919d5d8325ddfcb1d502e97f10ba512589e56abaf0de233a))


In [59]:
(s22, R2) = schnorr_sign(e, m2, k2)
print((s22,R2))

(101107292127574130836778977377221679966129011648684875611092108941656427460959, S256Point(1e1953f319bc2753b47231caaa25061bc6dc29b017eb01a848809ba70b12b0d2, 7aee36a5d36d5ad5919d5d8325ddfcb1d502e97f10ba512589e56abaf0de233a))


**Then, verify the generated signatures for the four cases 1.-4. using `ecdsa_verify`, and print the output.**


In [60]:
print(schnorr_verify(P, m1, s11, R1))
print(schnorr_verify(P, m2, s12, R1))
print(schnorr_verify(P, m1, s21, R2))
print(schnorr_verify(P, m2, s22, R2))

True
True
True
True


**c. What is the result of the signature verification of the output of setup 1. using the public nonce of the setup of 3.?** Explain your result.

In [62]:
print(schnorr_verify(P, m1, s11, R2))

False


The explanation is analogous to the part c) of the ECDSA exercise above.

#### c. (Leaking of Private Key after Reuse of Nonce)

Assume that a signer with access to the private key `e` publishes a valid signature Schnorr $(s_1,R_1)$ and $(s_2,R_2)$ for the two different messages `m1` and `m2`, respectively, generated using same the private nonce `k`.

**You are now an attacker that has access to $(s_1,R_1)$ and $(s_2,R_2)$ as well as the public key `P`. Can you obtain the private key from this information?**

Explain your answer and writing code that takes the signatures generated for `m1` and `m2` using `k1` above, as well as the public key `P` as input and returns `e`.

Print the output of this code.

In [66]:
print("Private Key:", e)
(s1, R1) = schnorr_sign(e, m1, k1)
(s2, R2) = schnorr_sign(e, m2, k1)
P = e*G

P_xonly = P.x.num.to_bytes(32, 'big')
R_xonly = R1.x.num.to_bytes(32, 'big')
m = m1.encode()
z1 = int.from_bytes(hash256(R_xonly + P_xonly + m), 'big') % N


R_xonly = R2.x.num.to_bytes(32, 'big')
m = m2.encode()
z2 = int.from_bytes(hash256(R_xonly + P_xonly + m), 'big') % N

z_inv = pow(z1 - z2, -1, N)

e_obtained = (s1-s2) * z_inv % N

print("Reconstructed Private Key:",e_obtained)

Private Key: 246835
Reconstructed Private Key: 246835
